In [1]:
import numpy as np 
import pandas as pd

import glob
import json

In [2]:
root_path = '/kaggle/input/CORD-19-research-challenge/'
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})
meta_df.head(2)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (1,5,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [3]:
all_json = glob.glob(f'{root_path}/**/pdf_json/*.json', recursive=True)
len(all_json)

#folder_contents = glob.iglob('subdir/*')
#for _, f in zip(range(100), folder_contents):
#    print(f)

401214

In [4]:
#pmc files are text only and do not contain abstract
#all_json_pmc = glob.glob(f'{root_path}/**/pmc_json/*.json', recursive=True)
#len(all_json_pmc)

In [5]:
meta_df.isnull().sum()

cord_uid                  0
sha                  682894
source_x                  0
title                   503
doi                  399880
pmcid                667089
pubmed_id            557728
license                   0
abstract             235542
publish_time           1814
authors               23869
journal               87322
mag_id              1056660
who_covidence_id     573725
arxiv_id            1042411
pdf_json_files       682894
pmc_json_files       740918
url                  369726
s2_id                 80192
dtype: int64

In [6]:
methods = ['methods','method','statistical methods','materials','materials and methods',
                'data collection','the study','study design','experimental design','objective',
                'objectives','procedures','data collection and analysis', 'methodology',
                'material and methods','the model','experimental procedures','main text']

In [7]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            self.methods = []
            self.results = []

            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            # Methods
            methods = ['methods','method','statistical methods','materials','materials and methods',
                'data collection','the study','study design','experimental design','objective',
                'objectives','procedures','data collection and analysis', 'methodology',
                'material and methods','the model','experimental procedures','main text']
            for entry in content['body_text']:
                section_title = ''.join(x.lower() for x in entry['section'] if x.isalpha()) #remove numbers and spaces
                if any(m in section_title for m in [''.join(x.lower() for x in m if x.isalpha()) for m in methods]) : 
                    self.methods.append(entry['text'])
            # Results
            results_synonyms = ['result']
            for entry in content['body_text']:
                section_title = ''.join(x.lower() for x in entry['section'] if x.isalpha())
                if any(r in section_title for r in results_synonyms) :
                    self.results.append(entry['text'])
                    
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
            self.methods = '\n'.join(self.methods)
            self.results = '\n'.join(self.results)

    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'
first_row = FileReader(all_json[0])
print(first_row)

8f97e16f3842e4bbd2d5d1c0c95ac1e31993ec68: In this short note, we apply the method of De Nicola et al. (2022) to the most recent available data, thereby providing up-to-date estimates of all-cause excess mortality in Germany for 2021. The anal... Expected deaths by year, represented by blue squares, plotted against observed fatalities, depicted by black dots. Overall excess mortality in 2021 was more pronounced than in 2020
In our article (De ...


In [8]:
dict_ = {'paper_id': [], 'abstract': [], 'body_text': [], 'methods': [], 'results': []}
for idx, entry in enumerate(all_json[0:40000]):
    if idx % (len(all_json[0:40000]) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json[0:40000])}')
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
    dict_['methods'].append(content.methods)
    dict_['results'].append(content.results)

Processing index: 0 of 40000
Processing index: 4000 of 40000
Processing index: 8000 of 40000
Processing index: 12000 of 40000
Processing index: 16000 of 40000
Processing index: 20000 of 40000
Processing index: 24000 of 40000
Processing index: 28000 of 40000
Processing index: 32000 of 40000
Processing index: 36000 of 40000


In [38]:
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text', 'methods', 'results'])
df_covid.head()

,paper_id,abstract,body_text,methods,results
0,8f97e16f3842e4bbd2d5d1c0c95ac1e31993ec68,"In this short note, we apply the method of De ...","Expected deaths by year, represented by blue s...",,
1,8187ea360c53a56ca2c579d758a5d6aa67716836,,Research Letter to the Editor:\nWe are writing...,,
2,a0d063dca746b135afe0451ce0b3bb1e06cf15ae,Background Brazil ranks second worldwide in to...,The COVID-19 pandemic has created an unprecede...,Our analysis is based on the SIVEP-Gripe publi...,The SIVEP-Gripe dataset comprised 99 557 patie...
3,edb294108440787c9f074483fd3c953a83e53622,,Die Corona-Pandemie ist eine Gefahr für die Ge...,,
4,e0777fb5df224525ee1b06008582b084c1b6b13b,"The burden of hypertension in South Africa, as...",Raised blood pressure (BP) and hypertension ar...,,"Of 156 participants attending the assessment, ..."


In [39]:
df_covid['abstract_word_count'] = df_covid['abstract'].apply(lambda x: len(x.strip().split()))
df_covid['body_word_count'] = df_covid['body_text'].apply(lambda x: len(x.strip().split()))
df_covid.head()

,paper_id,abstract,body_text,methods,results,abstract_word_count,body_word_count
0,8f97e16f3842e4bbd2d5d1c0c95ac1e31993ec68,"In this short note, we apply the method of De ...","Expected deaths by year, represented by blue s...",,,123,618
1,8187ea360c53a56ca2c579d758a5d6aa67716836,,Research Letter to the Editor:\nWe are writing...,,,0,801
2,a0d063dca746b135afe0451ce0b3bb1e06cf15ae,Background Brazil ranks second worldwide in to...,The COVID-19 pandemic has created an unprecede...,Our analysis is based on the SIVEP-Gripe publi...,The SIVEP-Gripe dataset comprised 99 557 patie...,375,4117
3,edb294108440787c9f074483fd3c953a83e53622,,Die Corona-Pandemie ist eine Gefahr für die Ge...,,,0,2421
4,e0777fb5df224525ee1b06008582b084c1b6b13b,"The burden of hypertension in South Africa, as...",Raised blood pressure (BP) and hypertension ar...,,"Of 156 participants attending the assessment, ...",244,2958


In [40]:
df_covid.describe(include='all')

,paper_id,abstract,body_text,methods,results,abstract_word_count,body_word_count
count,40000,40000,40000,40000,40000,40000.000000,40000.000000
unique,40000,28089,39952,16148,12102,NaN,NaN
top,8f97e16f3842e4bbd2d5d1c0c95ac1e31993ec68,,The nuclear receptor heterodimers of liver X r...,,,NaN,NaN
freq,1,11771,7,23799,27870,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,160.550500,3983.701950
std,NaN,NaN,NaN,NaN,NaN,195.220821,5042.969552
min,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000
25%,NaN,NaN,NaN,NaN,NaN,0.000000,1799.750000
50%,NaN,NaN,NaN,NaN,NaN,151.000000,3285.000000
75%,NaN,NaN,NaN,NaN,NaN,244.000000,5078.000000


In [41]:
df_covid.drop_duplicates(['abstract', 'body_text'], inplace=True)
df_covid.describe(include='all')

,paper_id,abstract,body_text,methods,results,abstract_word_count,body_word_count
count,39952,39952,39952,39952,39952,39952.000000,39952.000000
unique,39952,28089,39952,16148,12102,NaN,NaN
top,8f97e16f3842e4bbd2d5d1c0c95ac1e31993ec68,,"Expected deaths by year, represented by blue s...",,,NaN,NaN
freq,1,11750,1,23768,27840,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,160.452518,3977.824890
std,NaN,NaN,NaN,NaN,NaN,193.533966,5004.097423
min,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000
25%,NaN,NaN,NaN,NaN,NaN,0.000000,1800.000000
50%,NaN,NaN,NaN,NaN,NaN,151.000000,3285.000000
75%,NaN,NaN,NaN,NaN,NaN,244.000000,5076.000000


In [42]:
#df_covid['abstract'].replace('', np.nan, inplace=True)
#df_covid=df_covid.dropna(subset=['abstract'])
#df_covid

In [43]:
df_covid.describe(include='all')

,paper_id,abstract,body_text,methods,results,abstract_word_count,body_word_count
count,39952,39952,39952,39952,39952,39952.000000,39952.000000
unique,39952,28089,39952,16148,12102,NaN,NaN
top,8f97e16f3842e4bbd2d5d1c0c95ac1e31993ec68,,"Expected deaths by year, represented by blue s...",,,NaN,NaN
freq,1,11750,1,23768,27840,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,160.452518,3977.824890
std,NaN,NaN,NaN,NaN,NaN,193.533966,5004.097423
min,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000
25%,NaN,NaN,NaN,NaN,NaN,0.000000,1800.000000
50%,NaN,NaN,NaN,NaN,NaN,151.000000,3285.000000
75%,NaN,NaN,NaN,NaN,NaN,244.000000,5076.000000


In [44]:
df_covid.to_excel('df_covid.xlsx')

In [45]:
df_covid.iloc[2000].abstract[:500]

'Background: A vaccine against SARS-CoV-2 is of high urgency. Here the safety and immunogenicity induced by a DNA vaccine (INO-4800) targeting the full length spike antigen of SARS-CoV-2 are described. Methods: INO-4800 was evaluated in two groups of 20 participants, receiving either 1.0 mg or 2.0 mg of vaccine intradermally followed by CELLECTRA Ò EP at 0 and 4 weeks. Thirty-nine subjects completed both doses; one subject in the 2.0 mg group discontinued trial participation prior to receiving th'

In [46]:
df_covid.iloc[2000].body_text[:1000]

'After the original cases in China in late 2019, the Severe Acute Respiratory Syndrome Coronavirus 2 (SARS-CoV-2), the virus that causes CoronaVirus Disease, 2019 (COVID-19) spread quickly, the World Health Organization declared a pandemic on March 24th, 2020, and within 5 months, over 23 million cases of infection and over 800,000 deaths have been reported globally. COVID-19 typically manifests as a mild infection with symptoms that may include fever, dry cough, fatigue and headache [6] . The elderly and those with comorbid conditions are prone to experience more severe symptoms, including pneumonia and multiorgan disease. Severe and critically-ill COVID-19 patients requiring intensive care and invasive mechanical ventilation can quickly overwhelm hospitals [7] . Despite the increasing number of infections and deaths around the globe, most people remain vulnerable to infection. There is an urgent need for safe and effective vaccines. Many are in development including nucleic acid [8, 

In [47]:
df_covid.iloc[0].abstract[:500]

'In this short note, we apply the method of De Nicola et al. (2022) to the most recent available data, thereby providing up-to-date estimates of all-cause excess mortality in Germany for 2021. The analysis reveals a preliminary excess mortality of approximately 2.3% for the calendar year considered. The excess is mainly driven by significantly higher excess mortality in the 60-79 age group.\nZusammenfassung In diesem kurzen Beitrag wenden wir die Methode von De Nicola et al. (2022) auf die neueste'

In [48]:
df_covid.iloc[0].body_text[:1000]

'Expected deaths by year, represented by blue squares, plotted against observed fatalities, depicted by black dots. Overall excess mortality in 2021 was more pronounced than in 2020\nIn our article (De Nicola et al. 2022) in this issue, we presented a simple and novel method to compute excess mortality in a given calendar year while effectively taking the age structure of the population into account. We then applied our method to age-stratified mortality data to obtain estimates for general and age group-specific excess mortality for Germany in 2020, the first year of the COVID-19 pandemic. As we enter 2022, mortality figures from 2021 are starting to become available. With this short note, we thereby aim to provide the reader with up-to-date estimates of excess mortality for the second consecutive year of the pandemic. Mortality data are provided by the German Federal Statistical Office (Destatis 2022). Figures for 2021 are, at time point of submission of this note, not final, and num

In [49]:
#notes for pre processing
#language check to remove non-english articles
#does using abstract work as training for summary?
#show most frequent words used 
#do abstracts share the most frequent words as documents

In [54]:
#create training and test sets
df_covid['abstract'].replace('', np.nan, inplace=True)
train_df=df_covid.dropna(subset=['abstract'])
train_df

,paper_id,abstract,body_text,methods,results,abstract_word_count,body_word_count
0,8f97e16f3842e4bbd2d5d1c0c95ac1e31993ec68,"In this short note, we apply the method of De ...","Expected deaths by year, represented by blue s...",,,123,618
2,a0d063dca746b135afe0451ce0b3bb1e06cf15ae,Background Brazil ranks second worldwide in to...,The COVID-19 pandemic has created an unprecede...,Our analysis is based on the SIVEP-Gripe publi...,The SIVEP-Gripe dataset comprised 99 557 patie...,375,4117
4,e0777fb5df224525ee1b06008582b084c1b6b13b,"The burden of hypertension in South Africa, as...",Raised blood pressure (BP) and hypertension ar...,,"Of 156 participants attending the assessment, ...",244,2958
8,a85faaddad3715b80e53657eee1596d2ee8866a6,Strategies to increase the bio-functionality o...,"Bread, regardless of the type, production proc...",,,1084,3945
9,89afc7a375be1b21ee163ab5877961e19a501638,Vaccine hesitancy could become a significant i...,The death toll of COVID-19 cases and failed pa...,The present study is based on publicly availab...,"In the study sample, most were in the age grou...",250,3344
...,...,...,...,...,...,...,...
39991,1f71320e3751e0a4a7a4044ab35f0df95647bd34,Background The ongoing COVID 19 pandemic broug...,Mandibular fractures are most often treated by...,All the patients who were referred to our unit...,Sixty-three patients reported to the accident ...,189,2686
39993,0dd0af835ffe41c88c18aefb060ac7c34e0bb6bb,The COVID-19 pandemic has lead to the implemen...,The COVID-19 pandemic is still not over around...,The target of this research is students who ar...,,156,3814
39994,14e0640de2acb0f23458227db0d2f1a71c885a4b,There is a desire and need among minor-attract...,There is a growing body of research seeking to...,,"Owing to the descriptive nature of this paper,...",242,11130
39995,ffdc50d239f8e531dc159993723c0cbc176a632e,Multiplexing of samples in single-cell RNA-seq...,Recent advances in single-cell and single-nucl...,,,159,4081


In [55]:
test_df=df_covid[df_covid['abstract'].isna()]
test_df

,paper_id,abstract,body_text,methods,results,abstract_word_count,body_word_count
1,8187ea360c53a56ca2c579d758a5d6aa67716836,NaN,Research Letter to the Editor:\nWe are writing...,,,0,801
3,edb294108440787c9f074483fd3c953a83e53622,NaN,Die Corona-Pandemie ist eine Gefahr für die Ge...,,,0,2421
5,ee5af71875f2e77135974c75980ce22fff03e4f8,NaN,"Particularly in these pandemic times, appeals ...",,,0,2930
6,a0bc6bc5b8547b98a2d77b81ca81cb18fa1b7ee9,NaN,"To the editor, We read with great interest the...",,,0,97
7,af289740ecabb11a55c08f7a4b99ca2c8c9e746e,NaN,Coronavirus disease 2019 (COVID-19) is a novel...,,,0,5490
...,...,...,...,...,...,...,...
39989,ffa282107ce4584842a4025bb5cf72978b96dede,NaN,Trainee Well-Being: More Important Than Ever P...,,,0,997
39992,97d38bcca1eef74fa1a916e4787b0247f324558f,NaN,Poult Enteritis and Mortality Syndrome (PEMS) ...,,The effects of PEMS infection on body weights ...,0,2695
39996,b37bf3ab1d814f853a42cd8958d7390b42343bd2,NaN,To the Editor:\nWe read with interest the arti...,,,0,974
39997,2c70b8caf708c5c1b0c9b811a80179061274cff7,NaN,zeichen diagnostisch eine Rolle spielen. Am Ko...,,,0,5592


In [70]:
## for preprocessing
import re
import nltk
nltk.download('punkt')
import sys  
!{sys.executable} -m pip install contractions
import contractions

## create stopwords
lst_stopwords = nltk.corpus.stopwords.words("english")
## add words that are too frequent
lst_stopwords = lst_stopwords + ["cnn","say","said","new"]

## cleaning function
def utils_preprocess_text(txt, punkt=True, lower=True, slang=True, lst_stopwords=None, stemm=False, lemm=True):
    ### separate sentences with '. '
    txt = re.sub(r'\.(?=[^ \W\d])', '. ', str(txt))
    ### remove punctuations and characters
    txt = re.sub(r'[^\w\s]', '', txt) if punkt is True else txt
    ### strip
    txt = " ".join([word.strip() for word in txt.split()])
    ### lowercase
    txt = txt.lower() if lower is True else txt
    ### slang
    txt = contractions.fix(txt) if slang is True else txt   
    ### tokenize (convert from string to list)
    lst_txt = txt.split()
    ### stemming (remove -ing, -ly, ...)
    if stemm is True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_txt = [ps.stem(word) for word in lst_txt]
    ### lemmatization (convert the word into root word)
    if lemm is True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_txt = [lem.lemmatize(word) for word in lst_txt]
    ### remove Stopwords
    if lst_stopwords is not None:
        lst_txt = [word for word in lst_txt if word not in 
                   lst_stopwords]
    ### back to string
    txt = " ".join(lst_txt)
    return txt



[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [71]:
## apply function to both text and summaries
train_df["text_clean"] = train_df["body_text"].apply(lambda x: utils_preprocess_text(x, punkt=True, lower=True, slang=True, lst_stopwords=lst_stopwords, stemm=False, lemm=True))
train_df["y_clean"] = train_df["abstract"].apply(lambda x: utils_preprocess_text(x, punkt=True, lower=True, slang=True, lst_stopwords=lst_stopwords, stemm=False, lemm=True))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [73]:
train_df.head()

,paper_id,abstract,body_text,methods,results,abstract_word_count,body_word_count,text_clean,y_clean
0,8f97e16f3842e4bbd2d5d1c0c95ac1e31993ec68,"In this short note, we apply the method of De ...","Expected deaths by year, represented by blue s...",,,123,618,expected death year represented blue square pl...,short note apply method de nicola et al 2022 r...
2,a0d063dca746b135afe0451ce0b3bb1e06cf15ae,Background Brazil ranks second worldwide in to...,The COVID-19 pandemic has created an unprecede...,Our analysis is based on the SIVEP-Gripe publi...,The SIVEP-Gripe dataset comprised 99 557 patie...,375,4117,covid19 pandemic ha created unprecedented worl...,background brazil rank second worldwide total ...
4,e0777fb5df224525ee1b06008582b084c1b6b13b,"The burden of hypertension in South Africa, as...",Raised blood pressure (BP) and hypertension ar...,,"Of 156 participants attending the assessment, ...",244,2958,raised blood pressure bp hypertension wellknow...,burden hypertension south africa well successi...
8,a85faaddad3715b80e53657eee1596d2ee8866a6,Strategies to increase the bio-functionality o...,"Bread, regardless of the type, production proc...",,,1084,3945,bread regardless type production process geogr...,strategy increase biofunctionality staple food...
9,89afc7a375be1b21ee163ab5877961e19a501638,Vaccine hesitancy could become a significant i...,The death toll of COVID-19 cases and failed pa...,The present study is based on publicly availab...,"In the study sample, most were in the age grou...",250,3344,death toll covid19 case failed pandemic prepar...,vaccine hesitancy could become significant imp...


In [75]:
train_df.to_excel('train_df.xlsx')

In [76]:
test_df.to_excel('test_df.xlsx')